In [1]:
import os
import zipfile
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import normalize

%matplotlib inline

### Unzipping all zipped CSVs in a directory

In [2]:
data_path = '/Users/sameer/Desktop/UVA/Spring/ML/Project/Data/Project-3-Causality/Stock data/'
data_dump = data_path+str('2017_csv/')
folders_in_path = 2017 #list(range(2017, 2018)) # list of folders (years) in data path
extension = '.zip'
folder = 2017
#for folder in folders_in_path: # for each folder (year) in data path

In [3]:
#Data extraction code
def data_extract(stock_list,data_path,folder):
    folder_path = data_path + str(folder) + '/' # path with year
    for item in os.listdir(folder_path): # loop through all zip folders in folder (year)
        if item.endswith(extension): # check for .zip extension
            file_path = folder_path + item # get full file path
            zip_files = zipfile.ZipFile(file_path) # zip obj
            zip_files_info = zip_files.infolist() # list of files inside this zip folder

            for file in zip_files_info: # for every file inside zip folder
                if file.filename in ['{}{}'.format(a, '.csv') for a in stock_list]:
                    file.filename = item[:-len(extension)] + '_' + file.filename # add the date to file name
                    zip_files.extract(file, path=data_dump) # extract all files to data folder

### Function to read all files

In [4]:
def read_data(stock_name,files, sampling_time = '15T'):
    d = pd.DataFrame()
    for file in files:
        d1 = pd.read_csv(file)
        if d1.Ticker.unique() == stock_name:
            d = pd.concat([d,d1])
    d["moddate"] = d.Date.astype('str').str[0:4] + "-" + d.Date.astype('str').str[4:6] + "-" + d.Date.astype('str').str[6:9] + " "+ d.TimeBarStart.astype('str')
    d.moddate = pd.to_datetime(d.moddate)
    d[stock_name+'_VWP'] = d.VolumeWeightPrice
    d = d[['moddate',stock_name+'_VWP']]
    d.set_index('moddate',inplace=True)
    d = d.resample('s').interpolate(method = 'time').resample(sampling_time).asfreq()
    return d

In [5]:
def reshape_data(data_dump,stock_list,aggregation_time,n_lags):
    files = glob.glob(data_dump+ '*.csv')
    db = pd.DataFrame()
    for stk in stock_list:
        a1 = read_data(stk,files,aggregation_time)
        db = pd.concat([db,a1],axis=1)
    
    #Creating lagged terms for all stocks
    lags = range(1, n_lags)

    dbm = db.assign(**{
    '{}_t_{}'.format(col, t): db[col].shift(t)
    for t in lags
    for col in db})
    
    dbm.dropna(inplace=True)
    #Reordering the dataset 
    dbm = dbm[dbm.columns[::-1]]
    #Normalizing the dataset
    dbm = pd.DataFrame(normalize(dbm, axis = 0),index = dbm.index, columns = dbm.columns)
    return dbm

In [6]:
#Specifying values for function call
stock_list = ['AAPL','AMZN']
n_lags = 15
aggregation_time = str(n_lags)+'T'

In [7]:
#Extracting datasets
data_extract(stock_list,data_path,folder)

In [8]:
dbm = reshape_data(data_dump,stock_list,aggregation_time,n_lags)

In [9]:
dbm.head()

,AMZN_VWP_t_14,AAPL_VWP_t_14,AMZN_VWP_t_13,AAPL_VWP_t_13,AMZN_VWP_t_12,AAPL_VWP_t_12,AMZN_VWP_t_11,AAPL_VWP_t_11,AMZN_VWP_t_10,AAPL_VWP_t_10,...,AMZN_VWP_t_4,AAPL_VWP_t_4,AMZN_VWP_t_3,AAPL_VWP_t_3,AMZN_VWP_t_2,AAPL_VWP_t_2,AMZN_VWP_t_1,AAPL_VWP_t_1,AMZN_VWP,AAPL_VWP
moddate,,,,,,,,,,,,,,,,,,,,,
2017-01-03 07:30:00,0.004183,0.004122,0.004180,0.004131,0.004183,0.004126,0.004182,0.004126,0.004182,0.004126,...,0.004190,0.004131,0.004186,0.004134,0.004185,0.004130,0.004191,0.004136,0.004191,0.004129
2017-01-03 07:45:00,0.004180,0.004131,0.004183,0.004126,0.004182,0.004126,0.004182,0.004126,0.004182,0.004128,...,0.004186,0.004134,0.004185,0.004130,0.004191,0.004136,0.004191,0.004130,0.004191,0.004131
2017-01-03 08:00:00,0.004183,0.004127,0.004182,0.004126,0.004182,0.004126,0.004182,0.004128,0.004182,0.004126,...,0.004185,0.004130,0.004191,0.004136,0.004191,0.004130,0.004191,0.004131,0.004190,0.004133
2017-01-03 08:15:00,0.004182,0.004126,0.004182,0.004126,0.004182,0.004128,0.004182,0.004126,0.004183,0.004125,...,0.004191,0.004136,0.004191,0.004130,0.004191,0.004131,0.004190,0.004133,0.004191,0.004129
2017-01-03 08:30:00,0.004182,0.004126,0.004182,0.004128,0.004182,0.004126,0.004183,0.004125,0.004184,0.004126,...,0.004191,0.004130,0.004191,0.004131,0.004190,0.004133,0.004191,0.004129,0.004183,0.004129


In [10]:
dbm.to_csv('gc_db.csv',index=False)

In [11]:
dbm1 = pd.read_csv('gc_db.csv')
dbm1.head()

,AMZN_VWP_t_14,AAPL_VWP_t_14,AMZN_VWP_t_13,AAPL_VWP_t_13,AMZN_VWP_t_12,AAPL_VWP_t_12,AMZN_VWP_t_11,AAPL_VWP_t_11,AMZN_VWP_t_10,AAPL_VWP_t_10,...,AMZN_VWP_t_4,AAPL_VWP_t_4,AMZN_VWP_t_3,AAPL_VWP_t_3,AMZN_VWP_t_2,AAPL_VWP_t_2,AMZN_VWP_t_1,AAPL_VWP_t_1,AMZN_VWP,AAPL_VWP
0,0.004183,0.004122,0.004180,0.004131,0.004183,0.004126,0.004182,0.004126,0.004182,0.004126,...,0.004190,0.004131,0.004186,0.004134,0.004185,0.004130,0.004191,0.004136,0.004191,0.004129
1,0.004180,0.004131,0.004183,0.004126,0.004182,0.004126,0.004182,0.004126,0.004182,0.004128,...,0.004186,0.004134,0.004185,0.004130,0.004191,0.004136,0.004191,0.004130,0.004191,0.004131
2,0.004183,0.004127,0.004182,0.004126,0.004182,0.004126,0.004182,0.004128,0.004182,0.004126,...,0.004185,0.004130,0.004191,0.004136,0.004191,0.004130,0.004191,0.004131,0.004190,0.004133
3,0.004182,0.004126,0.004182,0.004126,0.004182,0.004128,0.004182,0.004126,0.004183,0.004125,...,0.004191,0.004136,0.004191,0.004130,0.004191,0.004131,0.004190,0.004133,0.004191,0.004129
4,0.004182,0.004126,0.004182,0.004128,0.004182,0.004126,0.004183,0.004125,0.004184,0.004126,...,0.004191,0.004130,0.004191,0.004131,0.004190,0.004133,0.004191,0.004129,0.004183,0.004129
